In [ ]:
# !pip install "openai>=1.0.0" "datasets>=2.0.0"
# !pip install datasets

# 1. Retrieving Data

In [ ]:
from datasets import load_dataset
dataset = load_dataset("allenai/ai2_arc", 'ARC-Challenge')

ARC-Challenge/train-00000-of-00001.parqu(…):   0%|          | 0.00/190k [00:00<?, ?B/s]

ARC-Challenge/test-00000-of-00001.parque(…):   0%|          | 0.00/204k [00:00<?, ?B/s]

ARC-Challenge/validation-00000-of-00001.(…):   0%|          | 0.00/55.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

In [ ]:
# train test val dimensions
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1119
    })
    test: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 1172
    })
    validation: Dataset({
        features: ['id', 'question', 'choices', 'answerKey'],
        num_rows: 299
    })
})


In [ ]:
# taking a look at the data
print(dataset["train"][0])
print(dataset["validation"][0])
print(dataset["test"][0])
dataset["train"].features

{'id': 'Mercury_SC_415702', 'question': 'George wants to warm his hands quickly by rubbing them. Which skin surface will produce the most heat?', 'choices': {'text': ['dry palms', 'wet palms', 'palms covered with oil', 'palms covered with lotion'], 'label': ['A', 'B', 'C', 'D']}, 'answerKey': 'A'}
{'id': 'Mercury_SC_407695', 'question': 'Juan and LaKeisha roll a few objects down a ramp. They want to see which object rolls the farthest. What should they do so they can repeat their investigation?', 'choices': {'text': ['Put the objects in groups.', 'Change the height of the ramp.', 'Choose different objects to roll.', 'Record the details of the investigation.'], 'label': ['A', 'B', 'C', 'D']}, 'answerKey': 'D'}
{'id': 'Mercury_7175875', 'question': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?', 'choices': {'text': ['Planetary density will decrease.', 'Planetary years will become longer.', 'Pla

{'id': Value('string'),
 'question': Value('string'),
 'choices': {'text': List(Value('string')), 'label': List(Value('string'))},
 'answerKey': Value('string')}

In [ ]:
import pandas as pd
# df are elite
df = pd.DataFrame(dataset["train"])
df.head()

,id,question,choices,answerKey
0,Mercury_SC_415702,George wants to warm his hands quickly by rubb...,"{'text': ['dry palms', 'wet palms', 'palms cov...",A
1,MCAS_2009_5_6516,Which of the following statements best explain...,"{'text': ['The refrigerator door is smooth.', ...",B
2,Mercury_7233695,A fold observed in layers of sedimentary rock ...,"{'text': ['cooling of flowing magma.', 'conver...",B
3,Mercury_7041615,Which of these do scientists offer as the most...,"{'text': ['worldwide disease', 'global mountai...",D
4,Mercury_7041860,A boat is acted on by a river current flowing ...,"{'text': ['west', 'east', 'north', 'south'], '...",B


#2. Accuracy Measurement

In [ ]:
val = dataset['validation']
print(val)

Dataset({
    features: ['id', 'question', 'choices', 'answerKey'],
    num_rows: 299
})


In [ ]:
def format_prompt(example):
  question = example['question']
  choices = example['choices']
  answer = example['answerKey']

  lines = [question]
  for i in range(len(choices['text'])):
    label = choices['label'][i]
    text = choices['text'][i]
    lines.append(f"{label}. {text}")
  return '\n'.join(lines)

In [ ]:
prompts = [format_prompt(ex) for ex in val]

In [ ]:
# sanity check
print(prompts[202])

Why would male insects be treated to prevent sperm production?
A. To increase the number of female insects
B. To reduce the total population of insects
C. To produce new species of insects
D. To prevent insects from mating


In [13]:
# from google.colab import userdata
# key = userdata.get(OPENAI_API_KEY)
import os
# MODEL_ID = userdata.get("MODEL_ID")
# os.environ["OPENAI_API_KEY"] = key

In [ ]:
from openai import OpenAI
client = OpenAI()
def call_model(prompt: str):
  response = client.responses.create(
      model = MODEL_ID,
      temperature = 0, # deterministic
      input = [
          {
              "role": "user",
              "content": prompt
          }
      ],
      include=["message.output_text.logprobs"]
  )
  content = response.output[0].content[0]
  # prediction = content.text.strip()
  prediction = next((c for c in content.text if c.isupper()), None) # trying to avoid issues with whitespace/non-answer output
  logprob = content.logprobs[0].logprob
  return prediction, logprob

In [ ]:
prediction, logprob = call_model(prompts[0])
print(prediction, logprob)

D -8e-06


In [ ]:
import math
val_df = pd.DataFrame(val)
predictions = []
accurate = []
confidence = []

for i, prompt in enumerate(prompts):
  pred, lp = call_model(prompt)
  predictions.append(pred)
  accurate.append(pred == val_df['answerKey'][i])
  confidence.append(math.exp(lp))

In [ ]:
print(predictions)

['D', 'C', 'D', 'A', 'B', 'C', 'C', 'C', 'C', 'D', 'C', 'D', 'A', 'B', 'A', 'C', 'B', 'B', 'A', 'A', 'B', 'B', 'D', 'B', 'A', 'A', 'C', 'B', 'D', 'A', 'B', 'B', 'D', 'D', 'D', 'A', 'A', 'B', 'D', 'D', 'C', 'D', 'D', 'B', 'C', 'B', 'C', 'C', 'A', 'A', 'C', 'C', 'A', 'A', 'D', 'A', 'B', 'B', 'D', 'A', 'D', 'A', 'A', 'D', 'D', 'B', 'C', 'C', 'C', 'A', 'C', 'B', 'D', 'C', 'B', 'B', 'A', 'A', 'C', 'C', 'A', 'D', 'D', 'A', 'A', 'A', 'B', 'C', 'A', 'B', 'D', 'A', 'D', 'B', 'A', 'B', 'A', 'A', 'B', 'A', 'B', 'C', 'B', 'C', 'A', 'C', 'C', 'C', 'B', 'B', 'C', 'A', 'D', 'C', 'B', 'A', 'D', 'B', 'C', 'D', 'B', 'B', 'D', 'D', 'D', 'A', 'D', 'A', 'C', 'A', 'B', 'C', 'D', 'A', 'C', 'D', 'C', 'A', 'B', 'B', 'A', 'D', 'A', 'D', 'B', 'C', 'A', 'B', 'B', 'B', 'D', 'D', 'A', 'A', 'C', 'C', 'A', 'C', 'B', 'C', 'A', 'A', 'B', 'C', 'C', 'A', 'B', 'A', 'C', 'B', 'A', 'D', 'B', 'B', 'C', 'B', 'C', 'A', 'A', 'D', 'C', 'B', 'A', 'B', 'D', 'C', 'B', 'D', 'A', 'C', 'B', 'C', 'D', 'C', 'C', 'C', 'C', 'A', 'A', 'A',

In [ ]:
val_df['predictions'] = predictions
val_df['o'] = accurate
val_df['confidence'] = confidence
accuracy = val_df['o'].mean()
print(f"{accuracy*100:.1f}%")
print(f"{accuracy:.8f}%")
# this was really slow ~3mins

88.0%
0.87959866%


#3. Calibration Analysis using Brier Score

In [ ]:
brier = ((val_df["confidence"] - val_df["o"]) ** 2).mean()
print(f"Brier score: {brier:.4f}")

Brier score: 0.1148


#4. Error Footprint

In [ ]:
threshold = 0.99
val_df["answered?"] = val_df["confidence"] >= threshold
answered_df = val_df[val_df["answered?"]]

selective_accuracy = (answered_df["o"].sum() / len(answered_df)) if len(answered_df) > 0 else 0
print(f"Selective Accuracy: {selective_accuracy*100:.1f}%")
coverage = val_df["answered?"].mean()
print(f"Coverage: {coverage*100:.1f}%")

Selective Accuracy: 90.9%
Coverage: 92.0%
